In [93]:
import os
# Disable tensorflow warning messages.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
import numpy as np
import tensorflow as tf

## Load the entire Mishnah, in Hebrew and English.

In [94]:
mishnah_path = r'/Users/lee/Judaism/Sefaria_Contest/sefaria_txt/Mishnah/'

directories = [x[0] for x in os.walk(mishnah_path)]
english = [i for i in directories if 'English' in i and not 'Commentary' in i] 
hebrew = [i for i in directories if 'Hebrew' in i and not 'Commentary' in i] 

mishnah_english = ''
mishnah_hebrew = ''

for direct in english:
    mishnah = open(direct + '/clean.txt', 'rb').read().decode(encoding='utf-8')
    mishnah_english += mishnah
    
for direct in hebrew:
    mishnah = open(direct + '/clean.txt', 'rb').read().decode(encoding='utf-8')
    mishnah_hebrew += mishnah

In [95]:
print('Length of Mishnah: {} characters'.format(len(mishnah_hebrew)))

Length of Mishnah: 1648170 characters


In [96]:
vocab = sorted(set(mishnah_hebrew))
print ('{} unique Hebrew characters'.format(len(vocab)))

english_vocab = sorted(set(mishnah_english))
print ('{} unique English characters'.format(len(english_vocab)))

57 unique Hebrew characters
92 unique English characters


In [97]:
print(vocab)

['\n', ' ', '"', "'", '(', ')', ',', '.', ':', ';', '<', '>', 'b', 'r', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֻ', 'ּ', 'ׁ', 'ׂ', 'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'ך', 'כ', 'ל', 'ם', 'מ', 'ן', 'נ', 'ס', 'ע', 'ף', 'פ', 'ץ', 'צ', 'ק', 'ר', 'ש', 'ת', '׳', '״']


In [6]:
print(english_vocab)

['\t', '\n', ' ', '!', '"', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}', '¼', '¾', '–', '‘', '’', '“', '”', '…', '\uf0b7']


## Remove vowels from the Mishnah?

In [67]:
vowels = ['ְ', 'ֱ', 'ֲ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֻ', 'ּ', 'ׁ', 'ׂ']

## Vectorize the text, and create mapping dictionaries.

In [98]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in mishnah_hebrew])

In [99]:
char2idx

{'\n': 0,
 ' ': 1,
 '"': 2,
 "'": 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '<': 10,
 '>': 11,
 'b': 12,
 'r': 13,
 'ְ': 14,
 'ֱ': 15,
 'ֲ': 16,
 'ֳ': 17,
 'ִ': 18,
 'ֵ': 19,
 'ֶ': 20,
 'ַ': 21,
 'ָ': 22,
 'ֹ': 23,
 'ֻ': 24,
 'ּ': 25,
 'ׁ': 26,
 'ׂ': 27,
 'א': 28,
 'ב': 29,
 'ג': 30,
 'ד': 31,
 'ה': 32,
 'ו': 33,
 'ז': 34,
 'ח': 35,
 'ט': 36,
 'י': 37,
 'ך': 38,
 'כ': 39,
 'ל': 40,
 'ם': 41,
 'מ': 42,
 'ן': 43,
 'נ': 44,
 'ס': 45,
 'ע': 46,
 'ף': 47,
 'פ': 48,
 'ץ': 49,
 'צ': 50,
 'ק': 51,
 'ר': 52,
 'ש': 53,
 'ת': 54,
 '׳': 55,
 '״': 56}

## Remove vowels from the Mishnah?

In [65]:
vowels = np.arange(8, 21)

In [66]:
vowels

array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

## Split the text up into training sets.

In [100]:
seq_length = 200
examples_per_epoch = len(mishnah_hebrew)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

ה
ַ
ח
ִ
ט


In [101]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [102]:
sequences

<BatchDataset shapes: (201,), types: tf.int64>

In [103]:
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'הַחִטִּים וְהַזּוּנִין אֵינָן כִּלְאַיִם זֶה בָזֶה. הַשְּׂעֹרִים וְשִׁבֹּלֶת שׁוּעָל, הַכֻּסְּמִין וְהַשִּׁיפוֹן, הַפּוֹל וְהַסַּפִּיר, הַפֻּרְקְדָן וְהַטֹּפֵחַ, וּפוֹל הַלָּבָן וְהַשְּׁעוּעִים, אֵינָם'
' כִּלְאַיִם זֶה בָזֶה: \nהַקִּשּׁוּת וְהַמְּלָפְפוֹן, אֵינָם כִּלְאַיִם זֶה בָזֶה. רַבִּי יְהוּדָה אוֹמֵר, כִּלְאָיִם. חֲזֶרֶת וַחֲזֶרֶת גַּלִּים, עֻלְשִׁין וְעֻלְשֵׁי שָׂדֶה, כְּרֵשִׁים וּכְרֵשֵׁי שָׂד'
'ֶה, כֻּסְבָּר וְכֻסְבַּר שָׂדֶה, חַרְדָּל וְחַרְדָּל מִצְרִי, וּדְלַעַת הַמִּצְרִי וְהָרְמוּצָה, וּפוֹל מִצְרִי וְהֶחָרוּב, אֵינָם כִּלְאַיִם זֶה בָזֶה: \nהַלֶּפֶת וְהַנָּפוּץ, וְהַכְּרוּב וְהַתְּרוֹבְת'
'וֹר, הַתְּרָדִים וְהַלְּעוּנִים, אֵינָם כִּלְאַיִם זֶה בָזֶה. הוֹסִיף רַבִּי עֲקִיבָא, הַשּׁוּם וְהַשּׁוּמָנִית, הַבָּצָל וְהַבְּצַלְצוּל, וְהַתֻּרְמוֹס וְהַפְּלַסְלוֹס, אֵינָן כִּלְאַיִם זֶה בָזֶה: \nו'
'ּבָאִילָן, הָאֲגָסִים וְהַקְּרֻסְתּוּמֵלִין, וְהַפְּרִישִׁים וְהָעֻזְרָדִים, אֵינָם כִּלְאַיִם זֶה בָזֶה. הַתַּפּוּחַ וְהַחַזְרָד, הַפַּרְסְקִים וְהַשְּׁקֵדִין, וְהַשִּׁזָּפִין וְה

## Define a function to map from input text to target text (just the input text shifted to the right by one character).

In [104]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [105]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'הַחִטִּים וְהַזּוּנִין אֵינָן כִּלְאַיִם זֶה בָזֶה. הַשְּׂעֹרִים וְשִׁבֹּלֶת שׁוּעָל, הַכֻּסְּמִין וְהַשִּׁיפוֹן, הַפּוֹל וְהַסַּפִּיר, הַפֻּרְקְדָן וְהַטֹּפֵחַ, וּפוֹל הַלָּבָן וְהַשְּׁעוּעִים, אֵינָ'
Target data: 'ַחִטִּים וְהַזּוּנִין אֵינָן כִּלְאַיִם זֶה בָזֶה. הַשְּׂעֹרִים וְשִׁבֹּלֶת שׁוּעָל, הַכֻּסְּמִין וְהַשִּׁיפוֹן, הַפּוֹל וְהַסַּפִּיר, הַפֻּרְקְדָן וְהַטֹּפֵחַ, וּפוֹל הַלָּבָן וְהַשְּׁעוּעִים, אֵינָם'


In [106]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 32 ('ה')
  expected output: 21 ('ַ')
Step    1
  input: 21 ('ַ')
  expected output: 35 ('ח')
Step    2
  input: 35 ('ח')
  expected output: 18 ('ִ')
Step    3
  input: 18 ('ִ')
  expected output: 36 ('ט')
Step    4
  input: 36 ('ט')
  expected output: 25 ('ּ')


## Create training batches.

In [107]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 50000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>

## Build the RNN model.

In [108]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [109]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                 batch_input_shape=[batch_size, None]),
                                 tf.keras.layers.GRU(rnn_units,
                                                     return_sequences=True,
                                                     stateful=True,
                                                     recurrent_initializer='glorot_uniform'),
                                 tf.keras.layers.Dense(vocab_size)])
    return model

In [110]:
model = build_model(vocab_size = len(vocab),
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

In [111]:
model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (64, None, 256)           14592     
_________________________________________________________________
gru_7 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_7 (Dense)              (64, None, 57)            58425     
Total params: 4,011,321
Trainable params: 4,011,321
Non-trainable params: 0
_________________________________________________________________


## Predict characters with untrained RNN.

In [112]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 200, 57) # (batch_size, sequence_length, vocab_size)


In [51]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([ 4,  8,  7, 47, 42, 16, 15,  2, 11, 46, 17, 23, 31,  3, 23, 14,  5,
       31, 30, 22, 21, 27, 36, 11, 37, 42,  9,  2, 19,  4, 24, 44,  1,  5,
       31, 41,  3, 12, 41, 38, 21, 34, 30, 38, 39, 11, 30, 26,  3, 24, 11,
        1, 43, 11,  7,  0, 46, 19, 10, 15, 34,  4, 23,  2, 34, 47, 32,  4,
       37, 11, 21,  7, 10,  9, 13, 36, 47, 17, 37, 15, 46, 43, 23, 15, 41,
       10, 11, 41, 26, 31, 11, 16,  0, 30, 15, 45, 18, 36,  2, 24,  4, 41,
       36, 13,  5, 25, 28, 44,  2, 14, 29, 32, 10, 27,  4, 34, 36, 35, 21,
       34, 14, 36, 46,  4, 25, 13, 19, 17, 47, 30, 22, 43, 16, 10, 17,  7,
       31, 40, 29, 13, 15,  6,  9, 41, 32, 27, 29,  9, 10, 28,  7, 17, 10,
        4, 44, 36,  3, 45, 10, 22,  5,  1, 35,  4, 14, 34, 45,  6, 14, 14,
        0, 36,  1,  3, 47, 39, 18, 37, 27,  9, 34, 37, 29, 26,  5, 16, 19,
        3, 18, 39, 31, 36, 34,  4, 30, 38, 34, 25, 14,  9, 40,  8, 41, 14,
        4, 40, 40, 46, 12, 40, 14, 43, 37, 34, 41, 40, 11, 10,  1, 32, 20,
       15, 44,  8, 46, 30

In [26]:
print('Input: \n', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Predictions: \n', repr(''.join(idx2char[sampled_indices])))

Input: 
 'ם לַעֲמֹד בְּשָׁלֹשׁ שָׁעוֹת. הַקּוֹרֵא מִכָּאן וְאֵילָךְ לֹא הִפְסִיד, כְּאָדָם הַקּוֹרֵא בַתּוֹרָה: \nבֵּית שַׁמַּאי אוֹמְרִים, בָּעֶרֶב כָּל אָדָם יַטּוּ וְיִקְרְאוּ, וּבַבֹּקֶר יַעַמְדוּ, שֶׁנֶּאֱמ'

Predictions: 
 "ֻזְֱִׂ\nרם'ְֲ\nט:דֻׂאחוו.חנ:'שזתלת,ֵמ,(חֹונףיָ(םהּׂ(םעןך(ס'שָאףךכ\nֱפׁ\nֶׁ.צרע'מַּׁאצטיכץעת,,ד.ַָ.ֲֲט)ְלַש ףְ.ְש,לְצָ..ָ ְםמףךאַבהמנכ(ֲטרֻץגכִ,ֻףֱז)ץצלֵ:ׁמץ.טְֵ,ף:ׁ\n\nֱזפה'זרּף.ָקאל.ב דקט:טשלּלמֲָ,נג:ֹץחוג"


## Train the model.

In [113]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 200, 57)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.04316


In [114]:
model.compile(optimizer='adam', loss=loss)

In [115]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS=30 #30

In [117]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
128/128 [==============================] - 270s 2s/step - loss: 2.0233
Epoch 2/30
128/128 [==============================] - 265s 2s/step - loss: 1.5842
Epoch 3/30
128/128 [==============================] - 271s 2s/step - loss: 1.3604
Epoch 4/30
128/128 [==============================] - 271s 2s/step - loss: 1.2386
Epoch 5/30
128/128 [==============================] - 270s 2s/step - loss: 1.1599
Epoch 6/30
128/128 [==============================] - 271s 2s/step - loss: 1.1023
Epoch 7/30
128/128 [==============================] - 270s 2s/step - loss: 1.0558
Epoch 8/30
128/128 [==============================] - 276s 2s/step - loss: 1.0179
Epoch 9/30
128/128 [==============================] - 290s 2s/step - loss: 0.9837
Epoch 10/30
128/128 [==============================] - 293s 2s/step - loss: 0.9530
Epoch 11/30
128/128 [==============================] - 293s 2s/step - loss: 0.9238
Epoch 12/30
128/128 [==============================] - 297s 2s/step - loss: 0.8947
Epoch 13/30
1

In [118]:
def generate_text(model, start_string, num_generate=2000, temperature=1.0):
    # Evaluation step (generating text using the learned model)
    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    # Empty string to store our results
    text_generated = []
    
    # Here batch size == 1
    model.reset_states()
    for i in np.arange(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
        
    return start_string + ''.join(text_generated)

In [120]:
fmodel = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
fmodel.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
fmodel.build(tf.TensorShape([1, None]))

In [121]:
print(generate_text(fmodel, u'מֵ'))

מֵר אֶת הַצֶּמֶר, פִּגּוּל, וְחַיָּבִין עָלָיו כָּרֵת. לֶאֱכֹל כַּחֲצִי זַיִת, כָּשֵׁר, שֶׁאֵין אֲכִילָה וְהַקְטָרָה מִצְטָרְפִין: 
הָאַחִין שֶׁחָל עוֹלָה קוֹדְמִין לְאָדָם, כְּדֵי שֶׁיָּבֹא אֵלִיָּהוּ. בְּנֵי הַיָּבָם וְעַל הַקֹּדֶשׁ, וּמַתְחִיל, מִשָּׂרֵם מִמֶּנָּה גָּלְבּוֹן, קוֹרִין אֶת שְׁיָרֵי הַדָּשֶׁן קַלְקָחַת עָלֶיהָ: 
אֵלּוּ טֹא בְעוֹלוֹת, וּבָהֲרוּסוֹ. וְהַמְחֻבָּרִין בַּקַּרְק עַל יְדֵי עַצְמוֹ, אֵינוֹ חַיָּב אֶלָּא אַחַת. שְׁבוּעַת הַפַּת, וּפָתוּחַ לִרְשׁוּת הָרַבִּים, עַד שֶׁתִּכָּנֵס לַחֲבֵרוֹ, הֲרֵי הַשֵּׁנִי לִתֵּן לָהּ כְּתֻבָּתָהּ שִׁבְעָה זִכְרֵי כֹהֲנִים וְהַגַּיְדְּנִין עָלֶיהָ פְּרוֹזְבּוּל, רוֹעֶה הַבָּנִים אֲחֻזָּה לִשְׂדֵהָה שְׁבִיעִית מֵעֲבֵרוֹ וְאֵין לוֹ חֲזָקָה, וְיֵשׁ אוֹמְרִים, עַל הַחַר הַנָּחָשׁ, מִיתָתָן זֶל מִזַּב תְּרוּמָה, וּמַת בַּמַּעֲשֵׂר, אֵינָהּ כְּבָת. אָמַר לוֹ רַבִּי יְהוּדָה. וְרַבִּי שִׁמְעוֹן מְטַהֵר, וַחֲכָמִים מְטַהֲרִין. הַצִּיד אָנִי. אָמְרוּ לוֹ, אִישִׁי כֹהֵן גָּדוֹל, רַבִּי יְהוּדָה אוֹמֵר, אַף בַּנּוֹדֵר, הֲרֵי הוּא מְטַמֵּא בְג

In [122]:
print(generate_text(fmodel, u'אֵ'))

אֵין קִנֵּי זוֹבוֹתֶבֶת בַּחוּץ, אֵינָן מִצְטָרְפִין. וּשְׁאָר כָּל הַדָּבָר, חוּץ מִן הַבְּכוֹר וְהַמַּעֲשֵׂר שֶׁנֶּעֱקָב. עָשׂוּ אֲבוֹתֵיהֶן עַל מַיִם וְלֹא מִצָּה דָמָה שֶׁעָלָיו לַחִי וְכָּהוּב: 
הָיוּ שְׁנַיִם, נְטוּלָה. אִם נִמְצְאוּ דִבְרֵיהֶן קְצֹרוֹת, פְּטוּרִין, מִפְּנֵי שֶׁאֵין דַּחַת יְקִימָהּ לְחוֹמָה, עַד שֶׁיַּעֲמִן דַּם הַפָּר, וְהַמַּרְחֶשֶׁת, וְהַחַלּוֹת, וְהָרַג בֶּן חֲנַנְיָן וּמוֹצְאוֹת בְּכֶסֶף מַעֲשֵׂר, וּפוֹסֵל אֶת הַמִּקְוֶה. נְקוּבַת הָאַלְמָנָה, וְנֶאֱכֶלֶת בְּיַד אֶחָד, וַחֲכָמִים אוֹמְרִים, אִם שִׁיֵּר קֹרָאַחַר שֶׁעֲשָׂאָהּ לְשֵׁם קַבָּלִין, שִׁעוּרוֹ בְרֹאשׁ הָאִילָנוֹת, טָהוֹר מִן הַמָּחוֹם, וְעַל הַדְּרָכִים, וְעַל הַדְּלָתוֹת, וְאֵינוֹ נֶאֱכָלוֹת אֶלָּא אַתָּה סֹגֶת. אֵיזֶהוּ עָרִיס, טַמִּין וְחֶצְיָן לְמַרְחִיקִין וְנִמְלְכוְ וְאֵינוֹ בְכוֹר לַנַּחֲלָה וְלֹא בְמֶלַח וְלֹא מְקַיֵּם, וְאֵינוֹ שָׁוֶה אֶלָּא בִשְׁבוּעָה. כְּדֵי גְמִי יָמִים לִכְלִי קִשּׁוּת אֶחָת. מַעְשַׂר עָנִי שֶׁיָבֹא לָצֵאת בְּכַבֵּיצָה, שֶׁבֵּינוֹ לְבַשֵּׁל בְּחָל עִם כְּרֹנָה. בַּעֲ

In [123]:
print(generate_text(fmodel, u'בְּ'))

בְּיָדָהּ וֹלַד שֶׁל יַיִן, אֵין לוֹ טוֹעֵם, כָּל הָרָעוּת אֵין לוֹקֵחַ אֶת הַזָּב, אֵין לִי בִנְכָסַיִךְ וּבַקָּשָׁה וְאֵין בֵּינִי לוֹקֶה. רַבִּי עֲקִיבָא אוֹמֵר, כַּמָּה אֶת הַשַּׁבָּת הַחֲמוֹר. הַחוֹבָל לָא וּלְנוּלָה וּמְחֻלְיָלָיו עַד סוֹף הָעוֹלָם. וְהַבַּיְדִּשְׁמֹנִים וְהָאֲבָנִים וְעָמַד בְּתוֹךְ שְׁלשִׁים וְאֶחָד וּמֵאֲמָנָה. מִן הַצַּד, אֵינוֹ נוֹהֵג אֶלָּא בְאוֹמֵר וִיק, בִּזְמַן שֶׁהֵן מֻרְחוֹק שִׁמְעוֹן אוֹמֵר, הַגַּנָּבִים שֶׁהִיא עוֹשָׂה אֶת הֶחָצֵר. לֹא יִטְבֹּל בָּאֶמְצַע. הָיְתָה בַתַּנּוּר, מַחֲזִיקִים יְדֵי חוֹבָתוֹ לַנָּשִׂיא אֶלָּא לְנִדָּה. הַמַּנִּיחַ אִסָּר שֶׁמָּד לִשְׁתֵּי הַלֶּחֶם, וְלֹא מִן הַמֻּקָּף וּצְרִיכָה גֵט וַחֲלֹב אִם תִּנָּשֵׂא וְלֹא תֵעָרַךְ בַּכַּלְיָט, וְהַמּוֹל, דִּבְרֵי רַבִּי מֵאִיר. רַבִּי יְהוּדָה אוֹמֵר, מֵבִיא וְקִיְּמָהּ. אֵין שְׁנֵיהֶם אֵינָן בְּרֹחַד מִתּוֹךְ שֶׁל זְקֵנָה, יַעֲלוּ וְלֹא תִקַּח מְטַהֵר אֶת יִשְׂרָאֵל: 
בָּא לָהֶם שֶׁיָּצָא מִן הַצְּדָדִין, קֻשְׁמָר לוֹ מִן קַבִּין, וּפוֹסֵל אֶת הַתְּרוּמָה וּמַעֲשֵׂר, דִּינֵי נְפָשׁו

In [124]:
print(generate_text(fmodel, u'עַל'))

עַל יְדֵי חוֹבָתוֹ. אָמַר רַבִּי אֱלִיעֶזֶר בֶּן צָלִין תַּחְתֶּיהָ, יַנִּיחֶנָּה לַחֲבֵרוֹ, עַד שֶׁיְּהֵא נִפְרָרִין וּשְׂעִירִים מַעֲשֵׂר שֵׁנִי שֶׁלָּהֶם וְיֵאָכְלוּ בֵיתָה אֵלּוּ מַצִּילִין זְרָעִים, אֵינָן חוֹשְׁשִׁים. וְאִם עֲשָׂאָן גִּדּוּלֵיהֶן, בִּשְׁלֹשָׁה. גֵּרְשָׁן הִיא הֲנָאָה, שֶׁנֶּאֱמַר (ויקרא ז), וְהִקְטִיר אֶת הַגַּנְטִילִין בִּסְקַדֵּשׁ מְשַׁלֵּם נֶזֶק שָׁלֵם, שֶׁאֵין אָדָם זֶה מְגַלֵּחַ חֲצִי נָזִיר, וְשָׂעִיר הַנִּשְׂרָפִין. רַבִּי יוֹסֵי מַתִּיר. וְכֻלָּן שֶׁעָשׂוּ מִשֶּׁרַבּוּ עַל לַחֲבֹת מִדִּבְרֵי, שׁוֹר שֶׁל תֶּלַם שֶׁל זְהוִּיתָתוֹ, הֲרֵי הוּא כְתוֹבָה וּלְסַפָּר כָּרָאוּי, מְעָרְבִין אֶחָדוֹת:  
שְׂכָר מַחֲלָהָה אֶל הַמִּדָּה. הַחוֹבָה מְלֻפֶּה מִשְׁקָלָהּ לְיִינָה, הִיא גְדוֹלָה, שְׁחִיטָתוֹ פְסוּלָה. רַבִּי יְהוּדָה אוֹמֵר, אֲפִלּוּ כָל הַנָּשִׂיא אוֹמֵר, אִם מֵעַכִּיר. אָמַר לוֹ רַבִּי עֲקִיבָא, אֲכִילַת הַיּוֹם, מִשֶּׁיַּבְשִׁיל. בֵּית שַׁמַּאי אוֹמְרִים, יוֹצִיאוּהוּזָה אֶת יָדָיו לְמַעְלָה מִן הַצִּיץ וְכוֹסְרִין, וּמַיִם בְּמַיִם שֶׁאֵין חָכְמָה. אִם 

In [126]:
print(generate_text(fmodel, u'ל'))

לֹכֶם מִן הַמֵּת, נוֹטֵל תְּשִׁיעֶזֶר אוֹמֵר, אֵין מְמִיחִין הֶעָרֵב. חָלְצוּ לְתוֹךְ שָׂדֶהוּ. אָכְלָהּ וְהִשְׁלִים בֵּין לַחֶבֶל, וְחָזְרָה אֶצְלָהּ. וּבֵית הִלֵּל מַתִּירִין: 
הַמַּעֲשֵׂר עָבְרוּ בֵית הִלֵּל לְבֵיתוֹ, אָבִינוּ עַל יְדֵי צָלוֹת אָבִיו, וְאֶת בְּנוֹ וּבֵית הַשְּׁלָחִין, וְהֶחָזֶרֶת וְחָפִין. הַלּוֹמֵד בַּחוּץ, וּבָהּ שְׁלֹשָׁה עַל שְׁלֹשָׁה מִלְמַעְלָן וְאֵינוֹ חַיָּב בָּהּ. אֵלּוּ הֵן מַשִּׂיאִין אֶת הָעָרְלָה, כִּי טַבַּעַת וְנַעֲשֵׂית עֲגֻלָּה, הָאוֹחֲזָיו מַרְאוֹת אוֹר אַחֲרֵיהֶם, אֶל יָחִיד, וָמֵת, לֹא עָכָה בָהֶן נָשִׁים, תֵּצֵא וְתִרְעֶה בָעֵדֶר. מִשֶּׁיַּסִּיקֶנּוּ, פְּטוּרִין. וְעַל קְטַנָּה בַת יִשְׂרָאֵל שֶׁנִּסֵּת לְיִשְׂרָאֵל, וְהִשִּׂיאוּ אַמְרֹאשׁ בִּזְמַן שֶׁהִיא לוֹקֶה אַרְבָּעִים. אֲבָל הוֹדוּ לוֹ חֲכָמִים. וְכֵן שְׁתֵּי גְרוֹנִין, אֶחָד בִּכְלִי אֶחָד, אֲפִלּוּ בְחֶזְקַת טֻמְאָה. הַחִיצוֹן מְחַלֶּה. הֻתַּר אֶחָד, כָּבוּשׁ שֶׁאֵין בָּהֶן כַּמָּה הֶדְיוֹט, הֶעֱבִירוֹ עַל גַּבֵּי שְׁלֹשָׁה, וּשְׁנֵי פַר דַּרְכִּיל, שָׂעִיר הַגִּתּוֹת טְהוֹרוֹת וְהֶחֱזִ

In [127]:
print(generate_text(fmodel, u'מַ'))

מְַּיָח, מַה הוּא. חַטַּאת הָעוֹף טָמֵא. הָיוּ שְׁנֵיהֶם שָׁוִין. אָמַר רַבִּי מֵאִיר, לֹא הָיִיתִי, הָעוֹלָה תִּקְרַב עוֹלָה, וְזֶה הַלּוּחַ בְּנֵי חַרְבַּיִם. בָּרִאשׁוֹנָה הָיוּ פְטוּרִין, דִּבְרֵי רַבִּי מֵאִיר. וַחֲכָמִים אוֹמְרִים, פְּעָמִים שֶׁהוּא בְחֶזְקָתָן, אֲפִלּוּ כְשֵׁרִים בְּמַשְׁקִים, מוֹחֲצִי אֶת דָּמוֹ. צָלְכֵי מָעוֹת, כְּדֵי לָמוּל בְּנַטּ, הֲרֵי זֶה יָכוֹל לַהֲסִיטָן, טְהוֹרִין. רַבִּי יְהוּדָה אוֹסֵר, הַמֵּת מְשַׁמֵּשׁ כְּנֶגֶד אֲרֻבָּה, הַבַּיִת טָמֵא. טֻמְאָה בַבַּיִת, מַה בּוֹ שֶׁל חֻלִּין, שֶׁנֶּאֱמַר (במדבר טז), וְנֶאֱכָלִים לִפְנִים מִן הַקְּלָעִים, קָרָאוּ לְאַחַר זְמָן. אָמַר רַבִּי עֲקִיבָא, אֲנִי אֲדַסֵּט בָּהּ. אָמְרוּ לוֹ אֱמֹר, אִם הָעוֹן עָשָׂר חֹדֶשׁ. מִשְׁפַּט אֶחָד, חַיָּב עַל הַשְּׁבוּעָה כִשְׁתֵּי הָאִילָן. אֲבָל עִמָּהּ טַעַם דָּבָר שֶׁדַּרְכּוֹ לְהַקְטִיר, דַּיָּהּ שְׁעָתָהּ. וְיֵשׁ אוֹכְלוֹת מַשְׁקִין יְבֵשִׁין, לְהָבִיא אֶת יְשָׁלָה שֶׁל יִשְׂרָאֵל, אָסוּר. קָדָשִׁים בִּירוּשָׁלַיִם, וְהָיוּ עֵדָיוָהָיְתָה בָאָרֶץ, וְחַלָּה נוֹהֵג בַּיָּבֵשׁ,

In [128]:
print(generate_text(fmodel, u'מַ'))

מַרְדֵּה. שָׁלֹשׁ עַל שָׁלֹשׁ שֶׁנִּתְנַסְּלוּ מְלַאכְתָּן שֶׁל מְצֹרָע, וּשְׂמֹאלוֹ וְאֵין בָּהּ יְדִיעָה, לֹא יָקֹב. וּבְחָרוּב וְהֶחֱזִיקוֹ, יִשָּׁפְכוּ לְתוֹךְ הַחֹלֶב: 
רַבִּי יְהוֹשֻׁעַ אוֹמֵר, בַּשְּׁבִיעִית מִנִּכְסֵי אִסָּרוֹת, וְלֹא סִימוֹן בְּזוֹחֲלִים, שֶׁהֵן כְּשֵׁרִים. וְנֶאֱכָלִין בְּשַׁבָּת, וְחֹמֶץ הָאֲמוּרָה בַתְּחִלָּה: 
הָאוֹנָאָה, אַף עַל פִּי שֶׁחֲשֵׁכָה, וּקְדֻשַּׁת הַשֵּׁם, וּתְהִאשֵׁר מִן הַמֵּת וְעֶצֶם כַּשְּׂעֹרָה. רַבִּי יְהוּדָה אוֹמֵר, אַף מִי שֶׁהָבִיא מִפְּנֵי שֶׁהֵן בִּרְשׁוּתוֹ שֶׁל אֲחֵרִים, וְאֶחָד לַתָּא מִן הַשִּׁיפָה בִּשְׁעַת הוֹצִיא, מוֹסִיף עוּר גֵּט וְאת לַאֲחֵרִים בְּבֵית הַפְּרָע. וְהָאוֹנִינִי נוֹהֵג בַּתְּרוּמָה. רָאָה אַחַת מַעֲשֵׂה: 
אֵלּוּ אֵינָן מְעֹרְבוֹת, הַמַּיִם שֶׁבְּתוֹךְ הֶקְדֵּשׁ, הַבָּשָׂר יֵאָכֵל: 
בַּחֲמִשָּׁה עָשָׂר בּוֹנְעִין בּמֵד. אָמְרוּ לוֹ, אִם כֵּן רִפָּנֵס הָיְתָה שָׁם בֵּית דִּין, שֶׁמַּצְנִיעִין הָאֵמוּם אֶחָד, אֲבָל רֹאשׁוֹ לַמּוֹסֵף רוּחַ חֲכָמִים. הַפֶּסַח שֶׁיֶּשׁ לָהּ וַאֲנִי אֶחֱר. אָמְרוּ לוֹ, מָמוֹן הָיְתָ